# Классификация тональности коротких текстов [SpaCy]

In [6]:
import spacy
import pandas as pd
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split


### Работа с датасетом

In [7]:
# Загрузить датасет
df = pd.read_csv('data/women_clothing.csv', encoding='utf8', sep='\t')
df


FileNotFoundError: [Errno 2] No such file or directory: '.data/women_clothing.csv'

In [ ]:
# убрать класс neutral
df = df[df['sentiment'] != 'neautral']


In [ ]:
# Разделить данные на обучающий и валидационный наборы
train_data, valid_data = train_test_split(df, test_size=0.3, random_state=42)


### list(tuple(text, label)) -> List(spacy.Doc.doc)

In [ ]:
! python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     --------------------------------------- 0.0/15.3 MB 660.6 kB/s eta 0:00:24
     --------------------------------------- 0.0/15.3 MB 393.8 kB/s eta 0:00:39
     --------------------------------------- 0.1/15.3 MB 819.2 kB/s eta 0:00:19
      --------------------------------------- 0.2/15.3 MB 1.1 MB/s eta 0:00:15
      --------------------------------------- 0.3/15.3 MB 1.1 MB/s eta 0:00:14
      --------------------------------------- 0.4/15.3 MB 1.4 MB/s eta 0:00:12
     - -------------------------------------- 0.4/15.3 MB 1.3 MB/s eta 0:00:12
     - -------------------------------------- 0.6/15.3 MB 1.7 MB/s eta 0:00:09
     -- ------------------------------------- 0.9/15.3 MB 2.2 MB/s eta 0:00:07
     --- ------------------------------------ 1.2/15.3 MB 2.8 MB/s eta 0:00:06
     --- ------------------------------------ 1.2/15.3 MB 2.8 MB/s eta 0:00:06
     ---- ----------------------------------- 1.6/15.3 M


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
nlp = spacy.load("ru_core_news_sm") # небольшой на русском

In [ ]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        # One Hot Encodding
        if label == 'negative':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [ ]:
train_docs = make_docs(train_data.values)

  0%|          | 0/42000 [00:00<?, ?it/s]

In [ ]:
valid_docs = make_docs(valid_data.values)

  0%|          | 0/18000 [00:00<?, ?it/s]

In [ ]:
# Запись на диск
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")

doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")


### Работа с SpaCy

https://spacy.io/usage/training#quickstart

base_config:

[paths]<br />
train = train.spacy<br />
dev = valid.spacy<br />
vectors = null<br />
[system]<br />
gpu_allocator = null<br />

[nlp]<br />
lang = "ru"<br />
pipeline = ["textcat"]<br />
batch_size = 1000<br />

[components]<br />

[components.textcat]<br />
factory = "textcat"<br />

In [ ]:
# на основе base_config сделать config.cfg
! python -m spacy init fill-config base_config.cfg config.cfg


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


config:

[training]<br />
dev_corpus = "corpora.dev"<br />
train_corpus = "corpora.train"<br />
seed = ${system.seed}<br />
gpu_allocator = \${system.gpu_allocator}<br />
dropout = 0.1<br />
accumulate_gradient = 1<br />
patience = 1600<br />
max_epochs = 2<br />
max_steps = 4000

In [ ]:
# тренируем модель
! python -m spacy train config.cfg --output ./output


ℹ Saving to output directory: \content\output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.005
E    #       LOSS TEXTCAT  CATS_SCORE  SCORE 
---  ------  ------------  ----------  ------
  0       0          0.25       60.50    0.60
  0     100         14.52       83.79    0.84
  0     200         13.49       85.10    0.85
  0     300         11.78       87.68    0.88
  0     400          9.87       88.74    0.89
  0     500          8.93       89.57    0.90
  0     600          7.68       90.00    0.90
  0     700          5.93       90.48    0.90
  0     800          6.51       90.58    0.91
  0     900          6.77       90.73    0.91
  0    1000          7.29       91.00    0.91
  0    1100          7.36       91.28    0.91
  0    1200          6.96       91.31    0.91
  0    1300        

### Тест модели

In [ ]:
# загружаем лучшую модель
nlp = spacy.load("output/model-best")
texts = ["Заказывала фиолетовую кофту размер XL, пришла очень маленькая кофта по длине 53 см и по груди тоже-детская разм. меньше чем S. Очень расстроилась, по спору присудили возврат денег 27 декабря. Выплата от 3 до 20 дней. Прошло уже 24 дня, деньги продавец не вернул, на сообщения не отвечает! Кофту такого смешного размера носить нельзя. Продавец непорядочный! Непонятно, почему Алиэкспресс после спора не отслеживают возвраты денег!",
         "Ужасная синтетика! Тонкая, ничего общего с представленной картинкой, не яркая, рисунок растянут и тусклый, впрочем как и сама кофта- мешок! На картинке кажется приталенной на самом деле нет! Не рекомендую",
         "Немного маловато,но жене понравилось",
         "Куртка без дефектов, без запаха, доставка месяц. Материал тонкий , просвечивает наполнитель.  На ог 108 ,  от. 86 размер xxx сел отлично."]

print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
for text in texts:
    print(text)
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] > .5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")


type : ‘quit’ to exit
Заказывала фиолетовую кофту размер XL, пришла очень маленькая кофта по длине 53 см и по груди тоже-детская разм. меньше чем S. Очень расстроилась, по спору присудили возврат денег 27 декабря. Выплата от 3 до 20 дней. Прошло уже 24 дня, деньги продавец не вернул, на сообщения не отвечает! Кофту такого смешного размера носить нельзя. Продавец непорядочный! Непонятно, почему Алиэкспресс после спора не отслеживают возвраты денег!
{'positive': 2.409719090934459e-09, 'negative': 1.0}
the sentiment is negative
Ужасная синтетика! Тонкая, ничего общего с представленной картинкой, не яркая, рисунок растянут и тусклый, впрочем как и сама кофта- мешок! На картинке кажется приталенной на самом деле нет! Не рекомендую
{'positive': 5.0845348596340045e-05, 'negative': 0.9999490976333618}
the sentiment is negative
Немного маловато,но жене понравилось
{'positive': 0.9905748963356018, 'negative': 0.009425117634236813}
the sentiment is positive
Куртка без дефектов, без запаха, достав